<a href="https://colab.research.google.com/github/luismiguelmartinluengo/PySpark_Demos/blob/main/Funciones_RDD.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import random
from pyspark.sql import SparkSession
from pyspark import SparkContext
from operator import add

In [ ]:


# Crear una sesión Spark
sparkSession = SparkSession.builder \
    .appName("MiSesionSpark") \
    .master("local[*]") \
    .getOrCreate()
# Verifica que la sesión se haya creado correctamente
print(sparkSession.version)
sparkContext = sparkSession.sparkContext
print(sparkContext)
#Otra forma de crear el contexto sin necesidad de declarar previamente una sesión:
#sparkContext = SparkContext("local", "Mi aplicacion con Contexto directo")
#Solo se puede tener un contexto activo a la vez

3.5.4
<SparkContext master=local[*] appName=MiSesionSpark>


In [ ]:
#flat vs flatmap
rdd = sparkContext.parallelize([1, 2, 3, 4, 5])
rddMap = rdd.map(lambda x: [x,x,x])
print(rddMap.collect())
rddFlatMap = rdd.flatMap(lambda x: [x,x,x])
print(rddFlatMap.collect())
rddSquared = rdd.map(lambda x: x**2)
print(rddSquared.collect())
rddMult = rdd.flatMap(lambda x: [x, x*10])
print(rddMult.collect())

[[1, 1, 1], [2, 2, 2], [3, 3, 3], [4, 4, 4], [5, 5, 5]]
[1, 1, 1, 2, 2, 2, 3, 3, 3, 4, 4, 4, 5, 5, 5]
[1, 4, 9, 16, 25]
[1, 10, 2, 20, 3, 30, 4, 40, 5, 50]


In [ ]:
#filter and union
rddFilter = rdd.filter(lambda x: x % 2 == 0)
print(rddFilter.collect())
rddUnion  = sparkContext.union([rdd, rddFlatMap, rddFilter])
print(rddUnion.collect())

[2, 4]
[1, 2, 3, 4, 5, 1, 1, 1, 2, 2, 2, 3, 3, 3, 4, 4, 4, 5, 5, 5, 2, 4]


In [ ]:
#reduce
rddSumatory = rdd.reduce(lambda x, y: x + y)
print(rddSumatory)

15


In [ ]:
#groupbykey
rddGroupedByKey = rdd.groupBy(lambda x: x % 3)
print(rddGroupedByKey.count())
for key, value in rddGroupedByKey.collect():
    print(key, list(value))

3
2 [2, 5]
0 [3]
1 [1, 4]


In [ ]:
#distinct y Sample
rddDistinct = rddFlatMap.distinct()
print(rddDistinct.collect())
#fraction 0.3 extraerá entre el 30 y el 40% de los datos
rddSample = rdd.sample(withReplacement=False, fraction=0.3, seed = None)
print(rdd.collect())
print(rddSample.collect())

[2, 4, 1, 3, 5]
[1, 2, 3, 4, 5]
[1]


In [ ]:
#Join inner/left/right
rddkv1 = sparkContext.parallelize([(1, [111,112]), (2, [121,122]), (3, [131,132]), (4, [141,142])])
rddkv2 = sparkContext.parallelize([(1, [211,212]), (2, [221,222]), (3, [231,232]), (5, [251,252])])
print(rddkv1.collect())
print(rddkv2.collect())
rddJoin = rddkv1.join(rddkv2)
print(rddJoin.collect())
rddLeftJoin = rddkv1.leftOuterJoin(rddkv2)
print(rddLeftJoin.collect())
rddRightJoin = rddkv1.rightOuterJoin(rddkv2)
print(rddRightJoin.collect())

[(1, [111, 112]), (2, [121, 122]), (3, [131, 132]), (4, [141, 142])]
[(1, [211, 212]), (2, [221, 222]), (3, [231, 232]), (5, [251, 252])]
[(1, ([111, 112], [211, 212])), (2, ([121, 122], [221, 222])), (3, ([131, 132], [231, 232]))]
[(4, ([141, 142], None)), (1, ([111, 112], [211, 212])), (2, ([121, 122], [221, 222])), (3, ([131, 132], [231, 232]))]
[(1, ([111, 112], [211, 212])), (5, (None, [251, 252])), (2, ([121, 122], [221, 222])), (3, ([131, 132], [231, 232]))]


In [ ]:
#union & subtract
rdd1 = sparkContext.parallelize([1,2,3,4,5])
rdd2 = sparkContext.parallelize([4,5,6,7,8])
rddUnion = rdd1.union(rdd2)
print(rddUnion.collect())
rddSubtraction = rdd1.subtract(rdd2)
print(rddSubtraction.collect())

[1, 2, 3, 4, 5, 4, 5, 6, 7, 8]
[1, 2, 3]


In [ ]:
#repartition & coalesce
print(rddJoin.getNumPartitions())
rddJoin = rddJoin.repartition(8)
print(rddJoin.getNumPartitions())
rddJoin = rddJoin.coalesce(3) #--> solo se puede reducir el número de particiones y para hacer reducción es más eficiente que repartition
print(rddJoin.getNumPartitions())

4
8
3


In [ ]:
#reduce & reduceByKey
rddReduce = rdd.reduce(add)
print(rddReduce)

letters = ['a','b','c','c','c','d','d','e']
rddLetters = sparkContext.parallelize(letters)
rddLetters = rddLetters.map(lambda x: (x,1))
print(rddLetters.collect())
rddLetters = rddLetters.reduceByKey(add)
print(rddLetters.collect())

15
[('a', 1), ('b', 1), ('c', 1), ('c', 1), ('c', 1), ('d', 1), ('d', 1), ('e', 1)]
[('b', 1), ('c', 3), ('d', 2), ('a', 1), ('e', 1)]


In [ ]:
#take & first
print(rddLetters.take(3))
print(rddLetters.first())
print(rddLetters.first())

[('b', 1), ('c', 3), ('d', 2)]
('b', 1)
('b', 1)


In [ ]:
#count
print(rdd.count())

5


In [ ]:
#for each --> no devuelve ningún resultado, es solo para aplicar algo sobre todos los elementos del RDD
def printElement(x):
  print(x) #--> esto no funciona porque el print se está haciendo en cada cluster no en el controlador
  #sirve para ejecutar tareas con los elementos del rdd pero que puedan ser ejecutadas desde cada uno de los nodos ejecutores.
#End printElement
rdd.foreach(printElement)

In [ ]:
#aggregate
#Esto se hace con tuplas, pero fuerzo un poco las cosas para hacerlo con un diccionario
acumuladorZero = {'suma':0, 'contador':0}
def acumulacion(acumulador, nuevoValor):
  suma = acumulador['suma'] + nuevoValor
  contador = acumulador['contador'] + 1
  return {'suma':suma, 'contador':contador}
#End acumulacion
def agregacion(acumulador1, acumulador2):
  suma = acumulador1['suma'] + acumulador2['suma']
  contador = acumulador1['contador'] + acumulador2['contador']
  return {'suma':suma, 'contador':contador}
#End agregacion
acumuladorFinal = rdd.aggregate(acumuladorZero, acumulacion, agregacion)
print(acumuladorFinal)
print('Promedio: ', str(acumuladorFinal['suma']/acumuladorFinal['contador']))

{'suma': 15, 'contador': 5}
Promedio:  3.0


In [ ]:
#saveAsTextFile --> ejecutado en local, genera una carpeta y dentro de ella los ficheros correspondientes a cada partición
rdd.repartition(4)
rdd.saveAsTextFile('save')

In [ ]:
#takeOrderer
#más eficiente que orderby y después extraer los n primeros elementos.
#takeOrderer no necesita ordenar todo el rdd como si hace orderby
print(rdd.takeOrdered(3))
print(rdd.takeOrdered(3, key = lambda x: -x))

[1, 2, 3]
[5, 4, 3]


In [ ]:
nu

In [ ]:
#histogram
rdd3 = sparkContext.parallelize(range(0,500))
rdd3 = rdd3.map(lambda x: x + random.randint(1,400))
bins, frecuencies = rdd3.histogram(3)
rdd3.coalesce(1)
max = rdd3.max()
min = rdd3.min()
print('Max: ', max)
print('Min: ', min)
print('Bins: ', bins)
print('Frecuencias: ', frecuencies)
binsManual = [min-10, ((max-min)/2)+min, max+10]
print(binsManual)
binsManual, frecuenciesManual = rdd3.histogram(binsManual)
print('Bins manual: ', binsManual, '; valores: ', sum(frecuencies))
print('Frecuencias manual: ', frecuenciesManual, '; valores: ', sum(frecuenciesManual))


Max:  865
Min:  27
Bins:  [32.0, 309.6666666666667, 587.3333333333334, 865]
Frecuencias:  [116, 269, 113]
[17, 446.0, 875]
Bins manual:  [17, 446.0, 875] ; valores:  498
Frecuencias manual:  [232, 266] ; valores:  498


In [ ]:
#sortby
rddSortBy = rdd3.sortBy(lambda x: x if x%2 == 0 else x+20) #los números impares avanzan 20 en el orden
print(rddSortBy.take(20))

[46, 50, 56, 41, 49, 70, 70, 51, 72, 80, 61, 65, 92, 83, 106, 112, 118, 101, 109, 109]


In [ ]:
#countbykey
rdd4 = sparkContext.parallelize(range(1,10))
keys = ['a','b','c','d']
rdd4 = rdd4.map(lambda x: (random.choice(keys), x))
print(rdd4.collect())
print(rdd4.countByKey().items())


[('a', 1), ('c', 2), ('d', 3), ('b', 4), ('c', 5), ('c', 6), ('a', 7), ('b', 8), ('d', 9)]
dict_items([('a', 4), ('b', 2), ('c', 2), ('d', 1)])


In [ ]:
#top --> devuelve los n primeros valores ordenados
print(rdd3.top(5))
print(rdd3.top(5, lambda x: -x))


[871, 870, 864, 858, 839]
[12, 21, 31, 40, 47]
